In [1]:
%load_ext autoreload
%autoreload 2

In [32]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import regex as re

In [50]:
CONFIG = [
    {
        "set_name" : "Dominaria United",
        "set_code" : "DMU",
        "URL" : "https://www.mtggoldfish.com/index/DMU-main_set#paper"
    },
    {
        "set_name" : "Double Masters 2022",
        "set_code" : "2X2",
        "URL" : "https://www.mtggoldfish.com/sets/Double+Masters+2022#paper"
    },
    {
        "set_name" : "Double Masters",
        "set_code" : "2XM",
        "URL" : "https://www.mtggoldfish.com/sets/Double+Masters#paper"
    },
]

In [57]:
raw_data_re = re.compile(r'(?<=\$\("\.price-sources-paper"\)\.toggle\(true\);\s+var d = "Date,.+";\s+)([^g]+)')
dates_re = re.compile(r'(\d{4}-\d{2}-\d{2})')
price_re = re.compile(r'(\d+\.\d+)')
def get_data(url):
    r = requests.get(test_url)
    soup = BeautifulSoup(r.text, "html.parser")
    scripts = soup.find_all("script", type="text/javascript", string=re.compile(r'var d = "Date'))

    if len(scripts) > 1:
        raise Exception('uh oh, more than one script found')
    script = scripts[0]

    raw_match = raw_data_re.findall(script.text)
    if len(raw_match) > 1:
        raise Exception('uh oh, more than one raw dataset found')
    raw_match = raw_match[0]

    dates = dates_re.findall(raw_match)
    prices = price_re.findall(raw_match)
    prices = [float(price) for price in prices]

    return dates, prices

In [58]:
test_url = "https://www.mtggoldfish.com/index/DMU-main_set#paper"
test_url = "https://www.mtggoldfish.com/sets/Double+Masters+2022#paper"
test_url = "https://www.mtggoldfish.com/sets/Double+Masters#paper"
dates, prices = get_data(test_url)

In [60]:
df = []
for set in CONFIG:
    dates, prices = get_data(set['URL'])
    temp_df = pd.DataFrame({'date': dates, 'price': prices})
    temp_df['set_name'] = set['set_name']
    temp_df['set_code'] = set['set_code']
    print(f"set_code: {set['set_code']}, set_name: {set['set_name']}")
    print(temp_df)
    df.append(temp_df)

df = pd.concat(df)

set_code: DMU, set_name: Dominaria United
           date    price          set_name set_code
0    2020-05-22   295.00  Dominaria United      DMU
1    2020-05-23   295.00  Dominaria United      DMU
2    2020-05-24   294.99  Dominaria United      DMU
3    2020-05-25   294.99  Dominaria United      DMU
4    2020-05-26   294.99  Dominaria United      DMU
..          ...      ...               ...      ...
917  2022-12-13  1904.74  Dominaria United      DMU
918  2022-12-14  1903.53  Dominaria United      DMU
919  2022-12-15  1902.11  Dominaria United      DMU
920  2022-12-16  1890.21  Dominaria United      DMU
921  2022-12-17  1896.10  Dominaria United      DMU

[922 rows x 4 columns]
set_code: 2X2, set_name: Double Masters 2022
           date    price             set_name set_code
0    2020-05-22   295.00  Double Masters 2022      2X2
1    2020-05-23   295.00  Double Masters 2022      2X2
2    2020-05-24   294.99  Double Masters 2022      2X2
3    2020-05-25   294.99  Double Masters 2022

In [61]:
df

,date,price,set_name,set_code
0,2020-05-22,295.00,Dominaria United,DMU
1,2020-05-23,295.00,Dominaria United,DMU
2,2020-05-24,294.99,Dominaria United,DMU
3,2020-05-25,294.99,Dominaria United,DMU
4,2020-05-26,294.99,Dominaria United,DMU
...,...,...,...,...
917,2022-12-13,1904.74,Double Masters,2XM
918,2022-12-14,1903.53,Double Masters,2XM
919,2022-12-15,1902.11,Double Masters,2XM
920,2022-12-16,1890.21,Double Masters,2XM
